# Practical 2 : Generative and Discriminative Models


In this practical, we will compare the Naïve Bayes Classifier (NBC) and Logistic Regression on several
datasets. As part of the practical you should read briefly the following paper:



**On Discriminative vs. Generative classifiers: A comparison of logistic regression
and naive Bayes**  
*Andrew Y. Ng and Michael I. Jordan*  
Advances in Neural Information Processing Systems (NIPS) 2001.

The paper is available on OLAT. 

You should read the Introduction and the Experiments sections. The goal of this practical is
to qualitatively reproduce some of the experimental results in this paper. You are strongly
encouraged to read the rest of the paper, which is rather short and straightforward to read,
though some of you may want to skip the formal proofs.

## Naïve Bayes Classifier (NBC)

You should implement a Naïve Bayes Classifier from scartch using NumPy. To keep your code tidy,
we recommend implementing it as a class. 
The classifier should be able to handle binary and continuous features. 
To earn the bonus points, the classifier should be able to handle categorical features as well. 
Suppose the data has 3
different features, the first being binary, the second being continuous and the third being categorical. Write an implementation that you can initialise as follows:

    nbc = NBC(feature_types=['b', 'r', 'c'])

Along the lines of classifiers provided in sklearn, you want to implement two more functions,
**fit** and **predict**. 
Recall the joint distribution of a generative model: $p(\mathbf{x}, y \mid \theta, \pi) = p(y \mid \pi) \cdot p(\mathbf{x} \mid y, \theta)$.
The **fit** function is to estimate all the parameters ($\theta$ and $\pi$) of the NBC, i.e., train the classifier. The **predict** function is to compute the probabilities that the new input belongs to all classes and
then return the class that has the largest probability, i.e., make the prediction.

    nbc.fit(X_train, y_train)
    ypredicted = nbc.predict(X_test)
    test_accuracy = np.mean(ypredicted == ytest)

Here we import the libraries. 

In [35]:
%matplotlib inline
import pylab
pylab.rcParams['figure.figsize'] = (10., 10.)

import pickle as cp
import matplotlib.pyplot as plt
import numpy as np
from copy import deepcopy, copy
import math


### Class-conditional distributions

Before implementing NBC, we first implement the class-conditional distributions $p(\mathbf{x} \mid y, \theta)$. Your implementation should have two functions: **estimate** and **get_log_probability**. 

- The **estimate** function takes data as input and models the data using some distribution $p(x \mid \theta)$, where $\theta$ is the parameters of this distribution. The function estimates the parameters $\theta$ using maximum likelihood estimators (MLE). 
For example, in the case of continuous features, we use the Gaussian distribution to model the data. The estimate function will find the parameters $\mu$ and $\sigma$ for the Gaussian distribution with respect to the input data. 

- The **get_log_probability** function takes as input a new data point $x_{new}$ and returns the log of $p(x_{new} \mid \theta)$. For the Gaussian distribution, the function get_probability will return $\mathcal{N}(x_{new} \mid \mu, \sigma)$. 

For different types of features, you need to use different distributions.
You can import statistic libraries (e.g., `scipy.stats`) for the implementation of the distributions. 

- For **continuous features**: Use Gaussian distribution
    https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.norm.html
- For **binary features**: Use Bernoulli distribution 
    https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.bernoulli.html
- For **categorical features**: Use Multinoulli distribution (The multinoulli distribution is a special case of the multinomial distribution, where the number of trials is 1)
    https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.multinomial.html



**Implementation Issues:**
- The probabilities can be very small. To avoid underflow issues, you should compute the log of the probabilities. Read more: (Mur) Chapter 3.5.3 / Lecture Notes
- The variance for Gaussian distributions should never be exactly 0, so in
case your calculated variance is 0, you may want to set it to a small value such as 1e − 6. This is to ensure that your code never encounters division by zero or
taking logarithms of 0 errors. 
For this practical, please set the small value to 1e-6.
- Laplace/Additive smoothing: You want to ensure that the estimates for the parameter for the Bernoulli and Multinoulli random variables is never exactly 0 or 1. For this reason you should consider using Laplace smoothing (https://en.wikipedia.org/wiki/Additive_smoothing).
For this practical, please set alpha to 1.
- We will check the correctness of your implementation using the tests below.
- For simplicity, you can assume the data values for binary features are integers from {0,1} and the data for a categorical feature with M categories are integers from {0, ..., M-1}.
- Fell free to add auxiliary functions or change the parameters of the functions. If you change the parameters of the functions, make sure you change the tests accordingly, so we can test your code.


In [63]:
from scipy.stats import norm, bernoulli, multinomial
import warnings

ALPHA = 1.0 # for additive smoothing

# Distribution for continuous features
class ContFeatureParam:
    def estimate(self, X):
        # TODO: Estimate the parameters for the Gaussian distribution 
        # so that it best describes the input data X
        # The code below is just for compilation. 
        # You need to replace it by your own code.
        ###################################################
        ##### YOUR CODE STARTS HERE #######################
        ###################################################
        mean, std = norm.fit(X)
        
        # std should never be 0 to prevent division by zero errors
        if std == 0:
          std = 10 ** -6
        
        self.theta = {"mean": mean, "std": std}
        ###################################################
        ##### YOUR CODE ENDS HERE #########################
        ###################################################

    def get_log_probability(self, X_new):
        # TODO: return the log of the density values for the input values X_new
        # The code below is just for compilation. 
        # You need to replace it by your own code.
        ###################################################
        ##### YOUR CODE STARTS HERE #######################
        ###################################################
        return norm.logpdf(X_new, self.theta["mean"], self.theta["std"])
        ###################################################
        ##### YOUR CODE ENDS HERE #########################
        ###################################################

        
        
def laplace_smoothing(X, dim, value, a=ALPHA):
    # count how often value appears in X ("pseudocount")
    amount_of_value_in_X = (X == value).sum()
    # return the smoothened probability
    return (amount_of_value_in_X + a) / (len(X) + a*dim)


# Distribution for binary features
class BinFeatureParam:
    def estimate(self, X):
        # TODO: Estimate the parameters for the Bernoulli distribution 
        # so that it best describes the input data X
        # The code below is just for compilation. 
        # You need to replace it by your own code.
        ###################################################
        ##### YOUR CODE STARTS HERE #######################
        ###################################################

        dim = 2 # 2 dimensions because we are dealing with binary features
        p = laplace_smoothing(X, dim, 1) # the probability of x=1
        # NOTE: the probability of x=0 would be 1 - p
        
        self.theta = {'p': p}

        ###################################################
        ##### YOUR CODE ENDS HERE #########################
        ###################################################

    def get_log_probability(self, X_new):
        # TODO: return the log of the probability values for the input values X_new
        # The code below is just for compilation. 
        # You need to replace it by your own code.
        ###################################################
        ##### YOUR CODE STARTS HERE #######################
        ###################################################
        return bernoulli.logpmf(X_new, self.theta['p'], loc=0)
        ###################################################
        ##### YOUR CODE ENDS HERE #########################
        ###################################################

# Distribution for categorical features
class CatFeatureParam:
    
    # we need to know the number of categories for the categorical feature
    def __init__(self, num_of_categories=0):
        self._num_of_categories = num_of_categories
    
    # setter method (needed because we initialize it with 0 by default)
    def set_num_of_categories(self, num_of_categories):
        self._num_of_categories = num_of_categories
        
    def estimate(self, X):
        # TODO: Estimate the parameters for the Multinoulli distribution 
        # so that it best describes the input data X
        # The code below is just for compilation. 
        # You need to replace it by your own code.
        ###################################################
        ##### YOUR CODE STARTS HERE #######################
        ###################################################
        
        probabilities = []
        for category in range(0, self._num_of_categories):
            dim = self._num_of_categories
            p = laplace_smoothing(X, dim, category)
            probabilities.append(p)
            
        # show warning if the probabilities do not sum up to 1
        if not math.isclose(np.sum(probabilities), 1, rel_tol=1e-5):
            warnings.warn(f'The probabilites do not sum up to 1! ({np.sum(probabilities)})')
        
        self.theta = {'p': probabilities}
        
        ###################################################
        ##### YOUR CODE ENDS HERE #########################
        ###################################################
        
    def get_log_probability(self, X_new):
        # TODO: return the log of the probability values for the input values X_new
        # The code below is just for compilation. 
        # You need to replace it by your own code.
        ###################################################
        ##### YOUR CODE STARTS HERE #######################
        ###################################################
        
        log_probs = []
        num_of_trials = 1 # multinoulli is a special case multinomial distribution with just 1 trial
        
        for category in X_new:
            x = np.zeros(self._num_of_categories)
            x[category] = 1
            prob = multinomial.logpmf(x, num_of_trials, self.theta['p'])
            log_probs.append(prob)
            
        return np.array(log_probs)
        ###################################################
        ##### YOUR CODE ENDS HERE #########################
        ###################################################

**Tests:**
    
We will use the code below to test the correctness of your code.

In [44]:
# continuous features

X = np.array([2.70508547, 2.10499698,1.76019132,3.42016431,3.47037973,3.67435061,1.84749286,4.3388506,2.27818252,4.65165335])

param = ContFeatureParam()
param.estimate(X)
param2 = ContFeatureParam()
probs = param.get_log_probability(np.array([0,1,3,4,5]))
print(probs)


[-5.64670664 -3.02757918 -0.90099277 -1.39353383 -2.92329773]


In [45]:
# binary features

X = np.array([0,0,1,1,0,1,0,1,1,1])

param = BinFeatureParam()
param.estimate(X)
probs = param.get_log_probability(np.array([0,1]))
print(probs)

[-0.87546874 -0.5389965 ]


In [81]:
# categorical features (bonus task)

X = np.array([0,6,5,4,0,6,6,4,1,1,2,3,8,8,1,6,4,9,0,2,2,3,8,0,2])

param = CatFeatureParam(num_of_categories=10)
param.estimate(X)
probs = param.get_log_probability(np.array([0,1,2,3,4,5,6,7,8,9]))
print(probs)

[-1.94591015 -2.1690537  -1.94591015 -2.45673577 -2.1690537  -2.86220088
 -1.94591015 -3.55534806 -2.1690537  -2.86220088]


### Implement NBC

We are now ready to implement NBC. We follow the structure of models in scikit-learn. We implement NBC as a class with functions **init**, **fit** and **predict**.
The **init** function takes as input the types of features and initialise the classifier. The **fit** function takes the training data as input and estimates the parameters. The **predict** function predicts the label for the input data. 

**Implementation Issues:**
- You should use matrix operations rather than loops. In general, loops over classes or features are OK, but loops over the rows of data are not a good idea.
- The probabilities can be very small. To avoid underflow issues, you should do the calculations in log space. Read more: (Mur) Chapter 3.5.3 / Lecture Note
- For simplicity, you can assume the data values for binary features are integers from {0, 1} and the data for a categorical feature with M categories are integers from {0, ..., M-1}.
- Fell free to add auxiliary functions or change the parameters of the functions. If you change the parameters of the functions, make sure you change the tests accordingly, so we can test your code.

In [96]:
# Your task is to implement the three functions of NBC. 

class NBC:
    # Inputs:
    #   feature_types: the array of the types of the features, e.g., feature_types=['b', 'r', 'c']
    def __init__(self, feature_types=[]):
        # TODO: 
        # The code below is just for compilation. 
        # You need to replace it by your own code.
        ###################################################
        ##### YOUR CODE STARTS HERE #######################
        ###################################################  
        
        self._feature_types = feature_types
        self._distributions = self.get_distributions_list() # list of distributions based on feature types
        
        ###################################################
        ##### YOUR CODE ENDS HERE #########################
        ###################################################

        
    def get_distributions_list(self):
        distributions = []
        for feature_type in self._feature_types:
            if feature_type == 'r':
                distribution = ContFeatureParam()
            elif feature_type == 'b':
                distribution = BinFeatureParam()
            elif feature_type == 'c':
                distribution = CatFeatureParam()
            distributions.append(distribution)
            
        return distributions
    
    def update_categorical_distributions(self):
        for idx, distribution in enumerate(self._distributions):
            if isinstance(distribution, CatFeatureParam):
                max_feat = np.max(X[:, idx])
                distribution.set_num_of_categories(max_feat + 1)
                
    def set_classes(self, y):
        self._classes = np.unique(y)   
        
    # The function uses the input data to estimate all the parameters of the NBC
    def fit(self, X, y):
        # TODO: 
        # The code below is just for compilation. 
        # You need to replace it by your own code.
        ###################################################
        ##### YOUR CODE STARTS HERE #######################
        ###################################################
        
        # we initialized all categorical distributions with num_of_categories=0 (because we lacked information)
        # at this point we know the values of X and we can update it accordingly
        self.update_categorical_distributions()
        
        # we can also figure out the unique classifications set
        self.set_classes(y)

        # dimensions of theta matrix
        n_classes = len(self._classes)
        n_features = len(self._distributions)
        
        self._pi = np.zeros(n_classes) # prepare list for pi values
        self._theta = np.full((n_classes, n_features), None) # prepare matrix for theta values
                
        for i, c in enumerate(self._classes):
            # calculate pi for each class
            self._pi[i] = (y == c).sum() / len(y)
            for j, distribution in enumerate(self._distributions): # [Cont, Cont, Bin, Cat]
                # select only rows that are classified with class c (y == c) and column of current feature (j)
                X_sub = X[y == c, j]
                distribution.estimate(X_sub)
                self._theta[i][j] = deepcopy(distribution)
        
        ###################################################
        ##### YOUR CODE ENDS HERE #########################
        ###################################################
                
            
    # The function takes the data X as input, and predicts the class for the data
    def predict(self, X):
        # TODO: 
        # The code below is just for compilation. 
        # You need to replace it by your own code.
        ###################################################
        ##### YOUR CODE STARTS HERE #######################
        ###################################################
        
        def predict_class(new_x):
            prob_classes = []
            
            # loop over every class
            for i, c in enumerate(self._classes):
                prob_c = np.log(self._pi[i]) # log-probability for class c
                
                # loop over every feature (resp. its distribution)
                for j, distribution in enumerate(self._theta[i]):
                    new_x_j = new_x[j] # the value of feature j of new_x
                    # compute the log-probability that the feature j of new_x belongs to class c ..
                    # .. and add it to the probability that new_x belongs to class c 
                    # (addition because we are dealing with log-values. otherwise it would be multiplication)
                    prob_c += distribution.get_log_probability([new_x_j])[0]
                    
                # at this point the value of prob_c is composed by the probabilities of each feature belonging to class c
                prob_classes.append(prob_c)                
            
            # select the highest posterior probability, indicating which class is most likely for new_x
            max_y_hat = np.argmax(prob_classes)
            return self._classes[max_y_hat]
        
        # predict class for every row of X
        y_hat = np.apply_along_axis(predict_class, 1, X)
        return y_hat
            
        ###################################################
        ##### YOUR CODE ENDS HERE #########################
        ###################################################


**Tests**

We will use the code below to check your code.

In [97]:
# All features of the iris dataset are continuous.

from sklearn.datasets import load_iris
iris = load_iris()
X, y = iris['data'], iris['target']

N, D = X.shape
Ntrain = int(0.8 * N)
Xtrain = X[:Ntrain]
ytrain = y[:Ntrain]
Xtest = X[Ntrain:]
ytest = y[Ntrain:]

nbc_iris = NBC(feature_types=['r', 'r', 'r', 'r'])
nbc_iris.fit(Xtrain, ytrain)
yhat = nbc_iris.predict(Xtest)
test_accuracy = np.mean(yhat == ytest)

print("Accuracy:", test_accuracy) # should be larger than 90%

Accuracy: 0.9333333333333333


In [94]:
# All features of this dataset are binary
import pandas as pd
data = pd.read_csv('binary_test.csv', header=None)
data = data.to_numpy()

X = data[:,1:]
y = data[:,0]

N, D = X.shape
Ntrain = int(0.8 * N)
Xtrain = X[:Ntrain]
ytrain = y[:Ntrain]
Xtest = X[Ntrain:]
ytest = y[Ntrain:]

nbc = NBC(feature_types=['b'] * 16)
nbc.fit(Xtrain, ytrain)
yhat = nbc.predict(Xtest)
test_accuracy = np.mean(yhat == ytest)

print("Accuracy:", test_accuracy) # should be larger than 85%

Accuracy: 0.8723404255319149


In [95]:
# All features of this dataset are categorical (bonus task)

data = pd.read_csv('categorical_test.csv', header=None)
data = data.to_numpy()

X = data[:,:-1]
y = data[:,-1]

N, D = X.shape
Ntrain = int(0.8 * N)
Xtrain = X[:Ntrain]
ytrain = y[:Ntrain]
Xtest = X[Ntrain:]
ytest = y[Ntrain:]


nbc = NBC(feature_types=['c'] * 9)
nbc.fit(Xtrain, ytrain)
yhat = nbc.predict(Xtest)
test_accuracy = np.mean(yhat == ytest)

print("Accuracy:", test_accuracy) # should be larger than 65% 

Accuracy: 0.6896551724137931


## Logistic Regression

For logistic regression, you should use the implementation in scikit-learn. Add the following
line to import the LR model.

In [98]:
from sklearn.linear_model import LogisticRegression

Check the scikit-learn documentation for the Logistic Regression model:
- http://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
- http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression


## Comparing NBC and LR

### Experiments

The experiment is to compare the classification error of the NBC and LR trained on increasingly larger training datasets. 
Since the datasets are so small, you should do this multiple times and
average the classification error. One run should look as follows:
- Shuffle the data, put 20% aside for testing.
    
    ```N, D = X.shape
    Ntrain = int(0.8 * N)
    shuffler = np.random.permutation(N)
    Xtrain = X[shuffler[:Ntrain]]
    ytrain = y[shuffler[:Ntrain]]
    Xtest = X[shuffler[Ntrain:]]
    ytest = y[shuffler[Ntrain:]]
    
    ```  


- Train the classifiers with increasingly more data. For example, we can train classifiers with 10%, 20%, ..., 100% of the training data. For each case store the classification errors on the test set of the classifiers.

You may want to repeat this with at least 200 random permutations (possibly as large as 1000)
to average out the test error across the runs. In the end, you will get average test errors as a function of the size of the training data. 
We have written for you the function for making the plots for the experiments. 

In [111]:
from sklearn.utils._testing import ignore_warnings
@ignore_warnings(category=RuntimeWarning)

# inputs:
#   nbc: Naive Bayes Classifier
#   lr: Logistic Regression Classifier
#   X, y: data
#   num_runs: we need repeat num_runs times and store average results
#   num_splits: we want to compare the two models on increasingly larger training sets.
#               num_splits defines the number of increasing steps. 
# outputs:
#   the arrays of the test errors across the runs of the two classifiers 

def compareNBCvsLR(nbc, lr, X, y, num_runs=200, num_splits=10):
    # The code below is just for compilation. 
    # You need to replace it by your own code.
    ###################################################
    ##### YOUR CODE STARTS HERE #######################
    ###################################################
    tst_errs_nbc = []
    tst_errs_lr = []
    
    rows = X.shape[0] # amount of data points
 
    Ntrain = int(0.8 * N) # we use 20% as test data
    print(Ntrain, 'Ntrain')
    shuffler = np.random.permutation(rows)
    # split into train and test data
    Xtrain = X[shuffler[:Ntrain]]
    ytrain = y[shuffler[:Ntrain]]
    Xtest = X[shuffler[Ntrain:]]
    ytest = y[shuffler[Ntrain:]]

    for split in range(num_splits):
        data_usage = (split + 1) * (100 / num_splits) / 100 # how much of the available training data is used (running from 0.1 to 1)
        print(f'Calculating errors by using {data_usage*100}% of the available training data')
        
        cur_tst_errs_nbc = [] # errors for NBC of the current split
        cur_tst_errs_lr = [] # errors for LogisticRegression of the current split

        for run in range(num_runs):
            # shuffle and select subset of training data depending on data_usage
            cur_Ntrain = int(data_usage * Ntrain)
            shuffler = np.random.permutation(Ntrain)
            cur_Xtrain = Xtrain[shuffler[:cur_Ntrain]]
            cur_ytrain = ytrain[shuffler[:cur_Ntrain]]

            # fit models
            nbc.fit(cur_Xtrain, cur_ytrain)
            lr.fit(cur_Xtrain, cur_ytrain)

            # predict on test data
            nbc_yhat = nbc.predict(Xtest)
            cur_tst_err_nbc = 1 - np.mean(nbc_yhat == ytest)
            lr_yhat = lr.predict(Xtest)
            cur_tst_err_lr = 1 - np.mean(lr_yhat == ytest)

            # append errors of this run
            cur_tst_errs_nbc.append(cur_tst_err_nbc)
            cur_tst_errs_lr.append(cur_tst_err_lr)

        # append mean errors over all runs
        tst_errs_nbc.append(np.array(cur_tst_errs_nbc).mean())
        tst_errs_lr.append(np.array(cur_tst_errs_lr).mean())

    return tst_errs_nbc, tst_errs_lr
    ###################################################
    ##### YOUR CODE ENDS HERE #########################
    ###################################################

In [105]:
def makePlot(nbc_perf, lr_perf, title=None, num_splits=10):
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)

    ax.tick_params(axis='both', labelsize=20)

    ax.set_xlabel('Percent of training data used', fontsize=20)
    ax.set_ylabel('Classification Error', fontsize=20)
    if title is not None: ax.set_title(title, fontsize=25)

    xaxis_scale = [(i + 1) * (100/num_splits) for i in range(num_splits)]
    plt.plot(xaxis_scale, nbc_perf, label='Naive Bayes')
    plt.plot(xaxis_scale, lr_perf, label='Logistic Regression', linestyle='dashed')
    
    ax.legend(loc='upper right', fontsize=20)

### Datasets

Tasks: For each dataset,
1. prepare the data for the two classifiers, e.g., handle missing values, handle text/categorical data, etc.
2. compare the two classifiers on the dataset and generate the plots
3. write a short report of how you prepare the data and your observations of the comparison 

**Dataset 1: Iris Dataset**

https://scikit-learn.org/stable/auto_examples/datasets/plot_iris_dataset.html

In [113]:
# TODO: insert your code for experiments
###################################################
##### YOUR CODE STARTS HERE #######################
###################################################
from sklearn.datasets import load_iris
iris = load_iris()

X, y = iris['data'], iris['target']

# 1. prepare the data for the two classifiers

# 2. compare the two classifiers on the dataset and generate the plots 
nbc = NBC(feature_types=['r', 'r', 'r', 'r'])
lr = LogisticRegression(max_iter=100000)
nbc_perf, lr_perf = compareNBCvsLR(nbc, lr, X, y)
makePlot(nbc_perf, lr_perf, title="Iris Dataset")

###################################################
##### YOUR CODE ENDS HERE #########################
###################################################

AttributeError: info

**Dataset 2: Voting Dataset**

https://archive.ics.uci.edu/ml/datasets/congressional+voting+records

The logistic regression line meets the naive bayes line early in the plot. To see it clearer, you should use only 100 data points from the dataset for this experiment.

In [ ]:
# load the dataset
# TODO: insert your code for experiments
###################################################
##### YOUR CODE STARTS HERE #######################
###################################################


voting = pd.read_csv('./voting.csv')

voting.info()

# 1. prepare the data for the two classifiers

# 1.1 handle missing data

# I would like to keep only those rows where at least 17-3=15 columns have finite values.
voting.dropna(thresh=(len(voting.columns)-3), inplace=True)

# drop irrelevant columns
voting.drop("water-project-cost-sharing", axis=1, inplace=True)
voting.drop("export-administration-act-south-africa", axis=1, inplace=True)

# fill remaining cells with most probable value
for column in voting:
    # print("---")
    missing_values = voting[column].isna()
    missing_values_sum = missing_values.sum()

    # print(missing_values_sum)

    if missing_values_sum != 0:
    props = voting[column].value_counts(normalize=True)
    # print("before props: " + str(props))
    pred_values = np.random.choice(props.index, p=props.values, size=missing_values_sum)
    voting.loc[missing_values, column] = pred_values
    # print("after props: " + str(voting[column].value_counts(normalize=True)))

voting.info()


# 1.2 handle text/categorial data
from sklearn.preprocessing import OrdinalEncoder

ordinal_encoder = OrdinalEncoder()
test = ordinal_encoder.fit_transform(voting)
np.unique(test, return_counts=True)

#voting.info()


# 1.3 shuffle data and pick 100 data points randomly
np.random.shuffle(test)
np.random.shuffle(test)

test = test[:100]

# 2. compare the two classifiers on the dataset and generate the plots 
X = test[:,1:]
y = test[:,0]
print(X.shape)
print(y.shape)
nbc = NBC(feature_types=['b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b'])
lr = LogisticRegression(max_iter=10000)
nbc_perf, lr_perf = compareNBCvsLR(nbc, lr, X, y)
makePlot(nbc_perf, lr_perf, title="Voting Dataset")



###################################################
##### YOUR CODE ENDS HERE #########################
###################################################